# Assignment 4



# Maxon Yin
# my2883




# Exercises





**1. Data Cleaning**

You will work with the same dataset as in Assigment 1.
The dataset has address

`url='https://github.com/amoreira2/Fin418/blob/main/assets/data/Assignment1.xlsx?raw=true'`

Do the followings:

- Import pandas, numpy, matplotlib, and load the data set.
- Import the datasets of industry returns and risk free rate.
- Parse the date.
- Set the index.
- Drop missing observations.
- Construct a dataframe with only excess returns.
- Call this dataframe with the 49 excess returns time series `df`.
- Call `df.head()` to check if everything works

**_Hint:_**
You did it in assignment 1 (the dataframe name was different), simply copy and paste your code.

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd

# your code below
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd

url = 'https://github.com/amoreira2/Fin418/blob/main/assets/data/Assignment1.xlsx?raw=true'
# Import the data
df_ind = pd.read_excel(
url,
sheet_name='49_Industry_Portfolios',
skiprows=6,
na_values=[-99.99, -999],
usecols="A:AX"
)
# Rename the column with date information
df_ind.rename(columns={df_ind.columns[0]: 'date'}, inplace=True)
# Convert the date column to datetime
df_ind['date'] = pd.to_datetime(df_ind['date'], format='%Y%m')
# Set date as the index
df_ind.set_index('date', inplace=True)

df_ind.index = df_ind.index.to_period("M").to_timestamp("M")

df_rmrf = pd.read_excel(
url,
sheet_name='Market_proxy',
skiprows=5,
na_values=[-99.99, -999],
)

df_rmrf.rename(columns={df_rmrf.columns[0]: 'date'}, inplace=True)
df_rmrf['date'] = pd.to_datetime(df_rmrf['date'], format='%Y%m')
df_rmrf.set_index('date', inplace=True)
df_rmrf.index = df_rmrf.index.to_period("M").to_timestamp("M")

def maybe_to_decimal(x):
    if isinstance(x, pd.Series):
        q = x.abs().quantile(0.95)
        return x/100 if pd.notna(q) and q > 2 else x
    else:
        q = x.abs().stack().quantile(0.95)
        return x/100 if pd.notna(q) and q > 2 else x

df_ind = maybe_to_decimal(df_ind)
rf = df_rmrf['RF'].astype(float)
rf = maybe_to_decimal(rf)

common_idx = df_ind.index.intersection(rf.index)
df_ind = df_ind.loc[common_idx].sort_index()
rf = rf.loc[common_idx].sort_index()

df = df_ind.subtract(rf, axis=0)
df = df.dropna(axis=0, how='any')

print("Excess-return dataframe shape:", df.shape)
display(df.head())





Excess-return dataframe shape: (553, 49)


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
date,,,,,,,,,,,,,,,,,,,,,
1969-07-31,-0.617100,-0.599600,-0.565400,-0.550700,-0.577600,-0.608000,-0.631900,-0.640600,-0.564600,-0.650500,...,-0.561400,-0.654900,-0.629400,-0.612800,-0.633500,-0.592700,-0.632000,-0.662600,-0.626600,-0.650200
1969-08-31,-0.454200,-0.454500,-0.445300,-0.433300,-0.460100,-0.436100,-0.384700,-0.399700,-0.452500,-0.474900,...,-0.438900,-0.444200,-0.436900,-0.417200,-0.411400,-0.377300,-0.412300,-0.345300,-0.409400,-0.424300
1969-09-30,-0.608600,-0.626100,-0.567800,-0.629300,-0.578600,-0.601300,-0.613600,-0.626500,-0.625400,-0.647000,...,-0.608200,-0.636100,-0.616600,-0.627500,-0.630200,-0.651100,-0.549900,-0.655200,-0.642900,-0.661900
1969-10-31,-0.437800,-0.520100,-0.531900,-0.483700,-0.475800,-0.535900,-0.469600,-0.525000,-0.536100,-0.526900,...,-0.565300,-0.552400,-0.488000,-0.537700,-0.510300,-0.492600,-0.543000,-0.562100,-0.498500,-0.592900
1969-11-30,-0.543000,-0.527200,-0.569200,-0.548200,-0.513900,-0.602800,-0.555200,-0.558900,-0.544300,-0.550900,...,-0.539600,-0.599300,-0.587100,-0.561200,-0.544700,-0.600600,-0.569000,-0.584400,-0.587000,-0.555600


**2. Expected excess return estimation**

Compute the sample mean as the estimators for the expected excess returns of the 49 assets.

Call this `ERe`.

**_Hint:_**
Use risk-free rates from the "Market_proxy" sheet.

In [10]:
# your code below

ERe = df.mean()

print("ERe (expected excess returns):")
ERe.head()

ERe (expected excess returns):


,0
Agric,-0.398391
Food,-0.396986
Soda,-0.396885
Beer,-0.396907
Smoke,-0.393629


**3. Expected excess return uncertainty**

We will now construct an estimator for the amount of uncertainty in our sample mean estimator. If we assume that each individual asset is uncorrelated over time (not terrible assumption), then the variance of the mean is

$$var(\bar{r_i}) = var(\frac{\sum_t^T r_{i,t}}{T})=\frac{\sum_t^T var( r_{i,t})}{T^2} = \frac{var(r_{i,t})}{T}$$

So all you need is the sample size (T) and the variance of each asset to obtain the varaince of our estimator.

Please use this formula to compute the **STANDARD DEVIATION** of sample average estimator of each 49 asset. Call this `ERe_se`.

In [11]:
# your code below

T = df.shape[0]
ERe_se = df.std() / np.sqrt(T)

print("Number of observations (T):", T)
print("ERe_se (standard error of mean estimator):")
ERe_se.head()

Number of observations (T): 553
ERe_se (standard error of mean estimator):


,0
Agric,0.012000
Food,0.011766
Soda,0.012095
Beer,0.011841
Smoke,0.011891


**4. Constructing a confidence interval for the expected excess return, part 1**

We will now want to construct the 95% confidence interval for our estimator. The interval is such that it contains the true mean 95% of the time.

The way to do this is to use the normal distribution CDF to figure out the threshold that leaves only 2.5% probability at the each side of the tails.

Why 2.5% and not 5%? Because it is symmetric, so there is 2.5% probability in the left tail and 2.5 % in the right tail so overall there is only 5% probability that the expected return is out of the interval. Thus there is 95% probability that it is in the interval.

In this exercise, you will find the threshold by doing the followings:

1. import the stats library from the scipy package with `from scipy import stats`

2. get the standard normal distribution with `sn=stats.norm(0,1)`, where 0 is the mean and 1 is the standard deviation

3. get the threshold by using inverse cumulative distribution function for the appropriate `prob_value` to create a 95% CI (see discussion above). `threshold=sn.isf(prob_value)`

4. make sure that this threshold is positive (if you got from the left tail, you will have to take the absolute value or just get from the right tail).

5. make sure you did things correctly by calling `print(threshold)`.

**_Hint:_**

You can always check in the normal table that you did things correctly https://en.wikipedia.org/wiki/Normal_distribution.


In [ ]:
# your code below

from scipy import stats

sn = stats.norm(loc=0, scale=1)
prob_value = 0.025
threshold = sn.isf(prob_value)

threshold = abs(threshold)

print("95% two-sided z-threshold:", threshold)

**5. Constructing a confidence interval for the expected excess return, part 2**

Armed with these threshold you can construct the interval as follows

$$[\bar{r}-threshold\times\sigma(\bar{r}),\bar{r}+threshold\times\sigma(\bar{r})]$$

Do the followings:

1. create an empty dataframe which has the names of industries as index and 'lower' and 'upper' as column names. Name it `ERe_ci`.

2. construct the lower bound of the interval, $\bar{r}-threshold\times\sigma(\bar{r})$, and store it in the column of 'lower'

3. compute the upper bound symmetrically, and store it in the column of 'upper'

4. call `ERe_ci.head()`

In [ ]:
# your code below

ERe_ci.head()

**6. Compute the tangency portfolio weights for a portfolio with annualized volatility of 10%**

A portfolio that is very important is the Mean-Variance-Efficient portfolio--sometimes also called the Tangency portfolio. This portfolio plays an outsized role in finance theory--for example any model of asset prices can be characterized as a prediction for which portfolio is MVE--for example in the case of the CAPM it predicts the market portfolio to be MVE.

For us we will simply discuss it's empirical implementation. It turns out that if you know the true moments of the excess returns distributions of the assets, $E[R^e]$  and $Var(R^e)$, then the portfolio that maximizes the SR is any portfolio weight proportional to

$$Var(R^e)^{-1}E[R^e]$$

So for any number x, the portfolio $W=xVar(R^e)^{-1}E[R^e]$ has the highest feasible sharpe ratio.

Let's construct this portfolio using the sample moments, so it will return the static portfolio that had the highest sharpe ratio in the sample.

1. you already estimated E[Re]
2. Now use .cov() to estimate the variance-covariance matrix and save it as `CovRe`
3. Invert the variance covariance matrix usign np.linalg.inv()
4. Construct the weights
5. Choose x so that the weights add up to 1

Store these in a dataframe whose rows are the names of the assets and the first column has the label 'mve_data'.

Name this data frame `Weights`.

`print(Weights)`



In [ ]:
# your code below

# Your CovRe
CovRe =

# Plug in appropriate variables in `invCovRe` and 'ERe'
raw_weights = 'invCovRe' @ 'ERe'

# Normalize weights to sum to 1
x =
norm_weights = x * raw_weights

# Store in DataFrame
Weights =

print(Weights.head())

**7. Sensitivity to uncertainty of the Tangent portfolio calculation**

Now we will compute the tangency portfolio but using a slightly different estimate for the mean.

Do the followings.

1. instead of using the sample mean for each asset, first we will pick one asset, `Hshld`.

2. change its mean to be its lower bound of CI (confidence interval) in Exercise 5 and then recalculate the tangency portfolio weights. (That is, change the mean of `Hshld` ONLY and go through the same process as exercise 6 to recalculate the portfolio weights for ALL assets.)

3. store this in dataframe `Weights` with the column name `mve_Hlth-1.95`.

4. create another column with weights computed from the perturbation in which the mean of `Hshld` is changed to be the upper bound of CI, label this column `mve_Hshld+1.95 `.

5. do a bar plot of these three sets of weights using Weights.plot.bar().

Discuss what you notice in the bar plot:

1. How much do the weights change?

2. Which assets are impacted? Why?

**_Hint:_**

you might want to create a copy of your ERe estimator before you do the perturbation.

In [ ]:
# your code below

# Make copies of `ERe` dataframe and name them `ERe_lower` and 'ERe_upper'
ERe_lower = ERe.copy()
ERe_upper = ERe.copy()

# Replace Hshld's mean with lower and upper CI bounds
ERe_lower['Hshld'] =
ERe_upper['Hshld'] =

# Compute weights as in the above exercise and save the values of each in `Weights` dataframe

# Bar plot

your discussion below

1. How much do the weights change?

2. Which assets are impacted? Why?


**8. Performance impact of estimation uncertainty**


Your `Weight` dataframe has 3 different weight schemes (three columns).

Do the followings:

1. compute the in-sample Sharpe Ratio for these 3 different weight schemes.

2. discuss the results that you obtained

**_Hint:_**
You should use the **real** data (e.g. `df`,`ERe`,`CovRe` ) to compute the Sharpe ratio (not the perturbed means in exercise 7).

In [ ]:
# your code below


your discussion below


**9. Reproduce analysis of Exercises 7-8 for all assets**

Do the followings:

1. use a for loop to loop through the 49 portfolios and create the "perturbed" weights and the Sharpe Ratio of the perturbed weights by performing the same exercises as in exercise 7 and 8.

2. record for each asset the average drop in the Sharpe Ratio associated with the perturbation in the tangency portfolio weights.

3. store the results in a dataframe named `dSR` (difference in SR):
$$dSR[asset]=\frac{1}{2}\frac{SR(asset+1.95)-SR+SR(asset-1.95)-SR}{SR(data)}$$
where SR(asset+1.95) and SR(asset-1.95) were the Sharpe ratios obtained when you perturb the expected excess return of that asset to the upper and lower bound of the CI.
`dSR` should be a dataframe with industry names as index and a column, called `SR_change`, containing the results of the calculation from the expression above.

4. do a bar plot of this Sharpe ratio change.

Discuss the bar plot:

1. What do you think is the key takeaway from the analysis above

***Hint:***
Note that all you need to do here is to get the code you developed above and adapt it to work with a for loop.

***Hint:***
When saving into `dSR`, You can use .from_dict method of pd.DataFrame


In [ ]:
# your code below

# It's useful to use dictionary for this exercise
sr_changes = {}

# For each asset, do the same exercise
for asset in ERe.index:
    # Write a code calculating Sharpe ratios using above exercise

    # Compute average SR drop. Hint: Use the formula in the problem.
    sr_diff =
    sr_changes[asset] = sr_diff

# Save it into DataFrame
dSR =

# Bar plot



your discussion below


**10. Monte Carlo, part 1**

So far our focus is on the estimation uncertainty of risk premiums. Covariance matrices also need to be estimated.

You will now implement a Monte-Carlo method to evaluate the overall uncertainty in the construction of the tangency portfolio.

You already have the sample estimates from the vector of expected excess returns `ERe` and the variance-covariance matrix `CovRe`  (you used those in Exercise 6).

Now you use the function `np.random.multivariate_normal` to simulate draws from a multivariate normal distribution with vector of mean equal to `ERe` and the covariance matrix equal to `CovRe`.

Do the following:

1. write the code that draws ONE realization of returns for this set of 49 assets.

***Hint:***

you should get a vector 49 by 1 that changes every time you run the cell.

type `np.random.multivariate_normal?` to see how this function works



In [ ]:
# your code below



**11. Monte Carlo, part 2**

Do the followings:

1. now set the parameter `size` in the `multivaraite_normal` function to draw T realizations of the 49 assets, where you set T to the number of months you have in the data set.

2. print the shape of your draw. This should return you a T by N matrix of returns, something with exactly the same shape as our data set.

***Hint:***
every time you run the cell again you get a different realization.



In [ ]:
# your code below



**12. Monte Carlo, part 3**

Do the followings:

1. copy the code above, so you have a simulated sample of monthly industry returns.

2. use the simulated return data and the weights in `mve_data` column of the dataframe `Weights` to construct a time-series of portfolio excess return. That is use the weights you constructed using the sample moments of the original sample.

3. compute and print its Sharpe Ratio.

***Hint:***

Every time you run this cell you should get a different Sharpe Ratio. This variation reflects the amount of overall uncertainty built in our investment strategy.

In [ ]:
# your code below
simulated_returns =
w_mve = Weights['mve_data'].values

# Compute time-series of simulated portfolio returns
simulated_port_returns =

# Compute Sharpe Ratio of the simulated portfolio

**13. Monte Carlo, part 4**

Now copy the code of the question above and write a for loop around it.

Do the followings:

1. loop throught this code 1000 times and each time record the resulting Sharpe Ratio.

2. save this in a dataframe called `MC`.

3. create a histogram of these Sharpe Ratios with 50 bins using the method `.hist`

Discuss the plot:

1. what do you conclude?

In [ ]:
# your code below

# Prepare saving space
mc_sharpes = []

# Run simulation 1000 times
for iter in range(1000):


your discussion below


**14. Please explain why an investor should care about these results.**

your answer below
